In [40]:
from pyspark.sql import SparkSession

In [56]:
spark = SparkSession.builder.appName('Practise').getOrCreate()

In [79]:
from pyspark.sql.functions import col,sum,avg,when

In [72]:
fis=spark.read.csv('fact_internet_sales.csv',header=True,inferSchema=True)
fis.show(2)

+----------+------------+-----------+-----------+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+
|ProductKey|OrderDateKey|CustomerKey|CurrencyKey|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount|  TaxAmt|Freight|
+----------+------------+-----------+-----------+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+
|       310|    20101229|      21768|         19|            1|  3578.27|       3578.27|                 0.0|           0.0|          2171.2942|       2171.2942|    3578.27|286.2616|89.4568|
|       346|    20101229|      28389|         39|            1|  3399.99|       3399.99|                 0.0|           0.0|          1912.1544|       1912.1544|    3399.99|271.9992|84.9998|
+----------+------------+-----------+--------

In [59]:
currency=spark.read.csv('dim_currency.csv',header=True,inferSchema=True)
currency.show(2)

+-----------+--------------------+--------------+
|CurrencyKey|CurrencyAlternateKey|  CurrencyName|
+-----------+--------------------+--------------+
|          1|                 AFA|       Afghani|
|          2|                 DZD|Algerian Dinar|
+-----------+--------------------+--------------+
only showing top 2 rows



In [60]:
customer=spark.read.csv('dim_customer.csv',header=True,inferSchema=True)
customer.show(2)

+-----------+---------+----------+--------+---------+----------+-------------+------+------+--------------------+------------+-------------+---------------+------------+-------------------+
|CustomerKey|FirstName|MiddleName|LastName|NameStyle| BirthDate|MaritalStatus|Suffix|Gender|        EmailAddress|YearlyIncome|TotalChildren|   AddressLine1|AddressLine2|              Phone|
+-----------+---------+----------+--------+---------+----------+-------------+------+------+--------------------+------------+-------------+---------------+------------+-------------------+
|      11000|      Jon|         V|    Yang|    false|1971-10-06|            M|  NULL|     M|jon24@adventure-w...|     90000.0|            2|3761 N. 14th St|        NULL|1 (11) 500 555-0162|
|      11001|   Eugene|         L|   Huang|    false|1976-05-10|            S|  NULL|     M|eugene10@adventur...|     60000.0|            3|     2243 W St.|        NULL|1 (11) 500 555-0110|
+-----------+---------+----------+--------+-------

In [61]:
date=spark.read.csv('dim_date.csv',header=True,inferSchema=True)
date.show(2)

+--------+--------------------+---------------+--------------------+--------------------+----------------+---------------+----------------+----------------+----------------+-----------------+---------------+------------+----------------+-------------+----------+--------------+
| DateKey|FullDateAlternateKey|DayNumberOfWeek|EnglishDayNameOfWeek|SpanishDayNameOfWeek|DayNumberOfMonth|DayNumberOfYear|WeekNumberOfYear|EnglishMonthName|SpanishMonthName|MonthNumberOfYear|CalendarQuarter|CalendarYear|CalendarSemester|FiscalQuarter|FiscalYear|FiscalSemester|
+--------+--------------------+---------------+--------------------+--------------------+----------------+---------------+----------------+----------------+----------------+-----------------+---------------+------------+----------------+-------------+----------+--------------+
|20050101|          2005-01-01|              7|            Saturday|              Sábado|               1|              1|               1|         January|          

In [70]:
product=spark.read.csv('dim_product.csv',header=True,inferSchema=True)
product.show()

+----------+-------------------+--------------------+------------------+------+---------+----+------+-----------------+-------+
|ProductKey|ProductAlternateKey|  EnglishProductName|SpanishProductName| Color|ListPrice|Size|Weight|DaysToManufacture| Status|
+----------+-------------------+--------------------+------------------+------+---------+----+------+-----------------+-------+
|         1|            AR-5381|     Adjustable Race|              NULL|    NA|     NULL|NULL|  NULL|                0|Current|
|         2|            BA-8327|        Bearing Ball|              NULL|    NA|     NULL|NULL|  NULL|                0|Current|
|         3|            BE-2349|     BB Ball Bearing|              NULL|    NA|     NULL|NULL|  NULL|                1|Current|
|         4|            BE-2908|Headset Ball Bear...|              NULL|    NA|     NULL|NULL|  NULL|                0|Current|
|         5|            BL-2036|               Blade|              NULL|    NA|     NULL|NULL|  NULL|   

In [101]:
full=(
fis
.join(date,fis.OrderDateKey==date.DateKey)
.filter(date.EnglishDayNameOfWeek=='Sunday')
.join(product,on='ProductKey',how='inner') 
.filter((product.Color=='Silver')&(product.Size.isNotNull())&(fis.Freight>10))
.join(customer,on='CustomerKey')
.filter((customer.YearlyIncome>100000)&(customer.TotalChildren > 1))
.drop('CustomerKey','OrderDateKey','ProductKey','CurrencyKey','AddressLine2','Suffix','MaritalStatus','NameStyle','MiddleName','SpanishProductName','FiscalQuarter','CalendarSemester','CalendarQuarter','SpanishMonthName','MonthNumberOfYear','WeekNumberOfYear','DayNumberOfYear','DayNumberOfMonth','SpanishDayNameOfWeek','EnglishDayNameOfWeek','DayNumberOfWeek')
)
full.show(2)



+-------------+---------+--------------+--------------------+--------------+-------------------+----------------+-----------+--------+-------+--------+--------------------+----------------+------------+----------+--------------+-------------------+--------------------+------+---------+----+------+-----------------+------+---------+---------+----------+------+--------------------+------------+-------------+----------------+-------------------+
|OrderQuantity|UnitPrice|ExtendedAmount|UnitPriceDiscountPct|DiscountAmount|ProductStandardCost|TotalProductCost|SalesAmount|  TaxAmt|Freight| DateKey|FullDateAlternateKey|EnglishMonthName|CalendarYear|FiscalYear|FiscalSemester|ProductAlternateKey|  EnglishProductName| Color|ListPrice|Size|Weight|DaysToManufacture|Status|FirstName| LastName| BirthDate|Gender|        EmailAddress|YearlyIncome|TotalChildren|    AddressLine1|              Phone|
+-------------+---------+--------------+--------------------+--------------+-------------------+----------

In [98]:
(
full
.groupBy(customer.FirstName)
.agg(sum(fis.TaxAmt).alias('TotalTaxAmt'),avg(fis.SalesAmount).alias('AvgSalesAmount'),avg(fis.TotalProductCost).alias('AvgTotalProductCost'))
.orderBy(col('FirstName').asc())
.show(2)
)

+---------+------------------+--------------+-------------------+
|FirstName|       TotalTaxAmt|AvgSalesAmount|AvgTotalProductCost|
+---------+------------------+--------------+-------------------+
|   Albert|          185.5992|       2319.99|          1265.6195|
|   Alexis|457.59839999999997|       2859.99|         1588.88695|
+---------+------------------+--------------+-------------------+
only showing top 2 rows

